# BLU10 - Learning Notebook - Part 2 of 3 - Rating Matrix

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp

from scipy.sparse import random, coo_matrix, lil_matrix, dok_matrix, csr_matrix, csc_matrix

# 1 Creating a ratings matrix

## 1.1 Community matrix

As we know, the community matrix represents our entire community in a single matrix.

Take $U = \{Ana, Miguel, Beatriz\}$, and $I = \{Bananas, Water, Milk\}$. 

We represent $U \times I$, aka the community matrix, as:

$$\begin{bmatrix}(Ana, Bananas) & (Ana, Water) & (Ana, Milk)\\ (Miguel, Bananas) & (Miguel, Water) & (Miguel, Milk)\\ (Beatriz, Bananas) & (Beatriz, Water) & (Beatriz, Milk)\end{bmatrix}$$

However, as we already know, the community matrix is not a thing.

## 1.2 Types of data

Users manifest their opinion about an item in different ways.

### Explicit and implicit feedback

Feedback is said to be explicit when provided by the user and implicit if inferred based on user actions (e.g., clicks).

Implicit feedback usually takes the form of unary data,

### Rating scale

We write $S$ the set of possible ratings. For example, in 1-5 stars rating system $r_{u, i} \in S = \{1, 2, 3, 4, 5\}$.

| Type of data    | Description                          | Rating scale (examples) | Explicit/Implicit |  
|-----------------|--------------------------------------|-------------------------|-------------------|
| Numeric         | Continuous ratings                   | $S = [1, 5]$            | Explicit          |
| Ordinal         | Ordered categories                   | $S = \{1, 2, 3, 4, 5\}$ | Explicit          |
| Binary          | Good or bad  (e.g., Upvote/Downvote) | $S = \{-1, 1\}$         | Explicit          |
| Unary           | User action  (e.g., Click, Purchase) | $S = \{1\}$             | Implicit          |
*Table 1: Different types of data and rating scales*

## 1.3 Ratings matrix

Consider the following ratings matrix $R$, with $S = \{1, 2, 3, 4, 5\}$:

$$\begin{bmatrix}1 &  & 2\\ 1 & 5 & \\  & 2 & 1\end{bmatrix}$$

## 1.4 Representing vectors

Let's go bit by bit, starting with the first row of the matrix, corresponding to:

$$\begin{bmatrix}(Ana, Bananas) & (Ana, Water) & (Ana, Milk)\end{bmatrix}$$

To clarify, $I_{Ana} = \{Bananas, Milk\}$ and $(Ana, Bananas) \notin R'$. Right?

At the core of Numpy is the homogeneous (i.e., all elements of the same type) n-dimensional array.

Corresponding to the NumPy array:

```
┌───┬───┬───┐
│ 1 │   │ 2 │
└───┴───┴───┘
```

We can create using `numpy.array` with an array-like object, a standard Python list in this case.

In [ ]:
np.array([1, np.NaN, 2])

The resulting array is what we call a rank-1 because it is a vector with one dimension.

Nonetheless, rank-1 arrays can be ambiguous, so we represent vectors as rank-2 arrays, i.e., as matrices with two dimensions.

The general convention is to use a column vector instead, i.e., a $n$ by 1 matrix, such as:

$$\begin{bmatrix}1 \\  \\ 2\end{bmatrix}$$

Corresponding to a 3 by 1 matrix, such as:

```
┌───┐
│ 1 │
├───┤
│   │
├───┤
│ 2 │
└───┘
```

We do this by using a list of lists, with one nested list per row.

In [ ]:
np.array([[1], [np.NaN], [2]])

## 1.5 Representing matrices

Finally, we create our ratings matrix $R$, corresponding to:
```
┌───┬───┬───┐
│ 1 │   │ 2 │
├───┼───┼───┤
│ 1 │ 5 │   │
├───┼───┼───┤
│   │ 2 │ 1 │
└───┴───┴───┘
```

Conveniently, we can pass a list of lists, just like we did above.

In [ ]:
R = [[1, np.NaN, 2], [1, 5, np.NaN], [np.NaN, 2, 1]]
R = np.array(R)
R

## 1.6 Matrix attributes

Some important attributes of any `ndarray`, to keep in mind.

In [ ]:
ndims = R.ndim
nrows = R.shape[0]
ncols = R.shape[1] 
dtype = R.dtype

print("R is a {}-dimensional, {} by {} matrix, of {} elements.".format(ndims, nrows, ncols, dtype))

## 1.7 Saving the matrix

We can save the matrix to a binary file in NumPy `.npy` format.

Note that `save` is a stand-alone function and not an array method.

In [ ]:
np.save('data/interim/ratings_matrix', R)

Alternatively, we can dump the matrix into a `.csv` file, as we would typically do.

In [ ]:
np.savetxt("data/interim/ratings_matrix.csv", R, delimiter=",")

# 2 Sparse Matrices

Huge matrices require much memory, and some large matrices are very sparse, i.e., with many more zero values than data.

This allocation is a waste of resources, as missing values and data cost the same space, but only the later hold any information.

In practice, this leads to matrices that don't fit in memory, despite having a manageable amount of data.

The premise of sparse data structures is that we *store only non-zero values*, and assume the rest of them are zeros.

**Sparse matrices** allow us to mitigate these problems:
* They are less memory-intensive, as they squeeze out the zeros and store only relevant values;
* Operations ignore zero values, i.e., the majority of the cells.

## 2.1 Sparse Matrices in SciPy

The `scipy.sparse` module implements sparse matrices based in regular NumPy arrays.

For the sake of objectivity, let's compare the sizes of a sparse versus a regular matrix.

We use `sp.sparse.random` to generate a sparse matrix of a given shape and density, with randomly distributed values.

(The term density is often used to refer to what we called the sparsity score previously.)

In [ ]:
def sparse_matrix_nbytes(M):
    return M.data.nbytes + M.indptr.nbytes + M.indices.nbytes


A = random(10 ** 3, 10 ** 5, density=.01, format='csr')
sparse_matrix_nbytes(A) / A.toarray().nbytes

So, there's that.

Let's explore how sparse matrices work and exemplify some implementations (more can be seen in the appendix).

### 2.1.1 Dictionary of Keys (DOK)

The most straightforward implementation of a sparse matrix is as a dictionary of keys, in which the keys are tuples represent indices.

```
┌───┬───┬───┐          
│ 2 │ 0 │ 0 │          {  
├───┼───┼───┤            (0, 0): 2,
│ 0 │ 5 │ 0 │ → DoK →    (1, 1): 5,
├───┼───┼───┤            (2, 1): 3,
│ 0 │ 3 │ 0 │          }
└───┴───┴───┘ 
```

In [ ]:
B = random(5, 5, density=.04, format='dok', random_state=42)

B.toarray()

In [ ]:
dict(B)

### 3.1.2 Compressed Sparse (CS)

Although the DOK implementation is quite easy to understand, the most used format is the **Compressed Sparse (CS)** and this is the one we are going to use going forward. It has a Row and a Column variants.

The **Compressed Sparse Row (CSR)**, uses three arrays:
* `data`, the value vector containing all non-zero values in [row-major order](https://en.wikipedia.org/wiki/Row-_and_column-major_order)
* `indptr`, the index pointer indicates at which element of the value vector the row starts
* `indices`, contains the column indices (which column each of the values come from).

```
┌───┬───┬───┐                   
│ 1 │ 0 │ 1 │          Matrix data:    [1, 1, 1] 
├───┼───┼───┤          
│ 0 │ 0 │ 0 │ → CSR →  Matrix indptr:  [0, 2, 2, 4]
├───┼───┼───┤          
│ 0 │ 0 │ 1 │          Matrix indices: [0, 2, 2]
└───┴───┴───┘ 
```

In fact, the index pointers tell us the starting and stopping indices `data[i, j]` for each row, above:
* The first row is given by `data[0:2]`
* The second row is given by `data[2:2]`
* The third row is given by `data[2:4]`.

The **Compressed Sparse Column (CSC)** format is similar, but the pointers refer to columns and the indices to the rows.

When comparing the two types of Compressed Sparse matrices:
* `CSR` provides efficient row slicing but slow column slicing, i.e., accessing and operating on row vectors
* `CSC` provides efficient column slicing but slow row slicing, i.e., accessing and operating on column vectors.

In [ ]:
E = random(5, 5, density=.2, format='csr', random_state=65)

E.toarray()

In [ ]:
E.data

In [ ]:
E.indptr

In [ ]:
E.indices

## 2.2 Creating Sparse Matrices

Back to our rating matrix $R$ from the previous unit, as:

```
    ┌───┬───┬───┐                   
    │ 1 │   │ 2 │
    ├───┼───┼───┤          
R = │ 1 │ 5 │   │
    ├───┼───┼───┤          
    │   │ 2 │ 1 │
    └───┴───┴───┘ 
```

In this section, we build sparse representations of $R$.

We start from our standard array.

In [ ]:
data = np.array([1, 0, 2, 1, 5, 0, 0, 2, 1]).reshape(3, 3)
data

### 2.2.1 DOK

The use-case for `DOK` is incremental construction.

In [ ]:
F = dok_matrix((3, 3))

for i in range(3):
    for j in range(3):
        F[i, j] = data[i, j]

F.toarray()

### 2.2.2 Compressed Sparse

Numpy matrices can easily be converted to the `CSR` format, so that we can efficiently operate on them.

In [ ]:
H_ = csr_matrix(data)
H_

In [ ]:
H_.data

In [ ]:
H_.indptr

In [ ]:
H_.indices

The process is exactly the same to convert to `CSC`.

In [ ]:
H_ = csc_matrix(data)
H_

### 2.2.3 From pandas DataFrame to scipy sparse

If you have a pandas DataFrame (containing only numerical values, of course), you don't need to create a numpy array from it and then convert to scipy sparse: you can do it directly!
This allows you to use Pandas to do cool feature engineering, plot some things and pretend you actually understand what the data is telling you.

In [ ]:
df = pd.DataFrame({
    'Bananas': [1,1,0],
    'Water': [0,5,2],
    'Milk': [2,0,1]
    },
    index=['Ana', 'Miguel', 'Beatriz']
)
df

In [ ]:
H_ = csc_matrix(df.values)
H_

In [ ]:
H_.toarray()